#derived from code shared by Meta AI. Many thanks to the Meta AI team for developing and maintaining this incredible Detectron2 library (and PyTorch itself!)

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
import torch, detectron2

In [ ]:
!pip install opencv-python
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
import matplotlib.pyplot as plt

def cv2_imshow(im):
    image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.show()

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)


In [ ]:
label_to_idx={'Femur':0,'Tibia':1}
from detectron2.structures import BoxMode

def get_leg_dicts(csv_path, fold):
    file=pd.read_csv(csv_path)
    file=file[file['fold']==fold]
    
    dataset_dicts = []
    unique_files = file['file'].unique()
    
    fileidx=0
    for thisfile in unique_files:
        temp = file[file['file']==thisfile]
        
        record={}
        record["file_name"] = "/home/appuser/detectron2_repo/SHARED/image/eos-image/" + thisfile
        record["image_id"] = fileidx
        record["height"] = temp['height'].iloc[0]
        record["width"] = temp['width'].iloc[0]
        
        objs = []
      
        for idx in range(0,len(temp)):
            poly=temp['points'].iloc[idx].replace("[","").replace("]","").split(",")
            poly = [float(x) for x in poly]
            px=poly[0::2]
            py=poly[1::2]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": label_to_idx[temp['label'].iloc[idx]]
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
        fileidx=fileidx+1
    return dataset_dicts

for d in ["train","tune","test"]:
    DatasetCatalog.register("leg_" + d, lambda d=d: get_leg_dicts("eoscsv.csv",d))
    MetadataCatalog.get("leg_" + d).set(thing_classes=["Femur","Tibia"])
    
leg_metadata = MetadataCatalog.get("leg_train")

In [ ]:
import pandas as pd

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader
import detectron2.utils.comm as comm
import torch
import time
import datetime

class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):            
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses
            
    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
        
        
    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)
        
class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)
                     
    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        return hooks

In [ ]:
import logging


cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("leg_train",)
cfg.DATASETS.TEST = ("leg_tune",)
cfg.TEST.EVAL_PERIOD =2000
cfg.SOLVER.CHECKPOINT_PERIOD=2000
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 1  
cfg.SOLVER.BASE_LR = 0.00025  
cfg.SOLVER.MAX_ITER = 100000    
cfg.SOLVER.STEPS = []       
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2 
cfg.INPUT.MIN_SIZE_TRAIN= (416,448,480,512,544,576,608)
cfg.INPUT.MAX_SIZE_TRAIN= 2560

cfg.INPUT.MIN_SIZE_TEST= 512
cfg.INPUT.MAX_SIZE_TEST= 2560

cfg.INPUT.RANDOM_FLIP="horizontal"

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = MyTrainer(cfg) 
trainer.resume_or_load(resume=False)